In [1]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
import tensorflow as tf

tflib.init_tf()
url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
    _G, _D, Gs = pickle.load(f)


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
image_path = './results/face.png'
image_size = (1024, 1024)
learning_rate = 0.001
num_iter = 1000

# Create target image variable 
image = PIL.Image.open(image_path)
image.resize(image_size, PIL.Image.ANTIALIAS)
image = np.array(image, dtype=np.float32)
with tf.variable_scope("recovery_scope", reuse=tf.AUTO_REUSE):
    target_image_variable = tf.get_variable('target_image', 
                                            dtype=tf.float32,
                                            initializer=tf.constant(image))
print(target_image_variable)
target_variable = tf.expand_dims(target_image_variable, 0)
# target_variable = tflib.convert_images_from_uint8(target_variable, drange=[-1,1], nhwc_to_nchw=True)
print(target_variable)

# Create initial latent variable 
# z = tf.Variable(np.random.randn(1, Gs.input_shape[1]), dtype=tf.float32)
with tf.variable_scope("recovery_scope", reuse=tf.AUTO_REUSE):
    z = tf.get_variable('learnable_latents',
                        shape=(1, Gs.input_shape[1]),
                        dtype=tf.float32,
                        initializer=tf.initializers.random_normal())
print(z)

# Create output image variable
# output_variable = Gs.get_output_for(z, None, is_validation=True, randomize_noise=False)
output_variable = Gs.get_output_for(z, None, truncation_psi=0.7, randomize_noise=False)
# formatted_output_variable = tflib.convert_images_to_uint8(output_variable, drange=[-1,1], nchw_to_nhwc=True)
formatted_output_variable = tflib.convert_images_to_uint8(output_variable, 
                                                          drange=[-1,1], 
                                                          nchw_to_nhwc=True, 
                                                          uint8_cast=False)
formatted_output_variable_uint8 = tf.saturate_cast(formatted_output_variable, tf.uint8)


print(output_variable)
print(formatted_output_variable)

# Loss
# loss = tf.losses.mean_squared_error(labels=target_variable, predictions=output_variable)
loss = tf.losses.mean_squared_error(labels=target_variable, predictions=formatted_output_variable)
print(loss)

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss, var_list=z)
print(optimizer)
print(train_op)

# Session
sess = tf.get_default_session()
# sess.run(tf.global_variables_initializer())
sess.run(z.initializer)
sess.run(target_image_variable.initializer)
formatted_output_value_uint8 = None
z_value = None
for it in range(num_iter):
    _, loss_value, z_value, formatted_output_value_uint8, formatted_output_value, target_value = sess.run((train_op, loss, z, formatted_output_variable_uint8, formatted_output_variable, target_variable))
    print("-------")    
    print(it)
    PIL.Image.fromarray(formatted_output_value_uint8[0], 'RGB').save('./results/recovery/{}.png'.format(it))
    print("Loss: ", loss_value)
#     print("formatted_output_value: ", formatted_output_value)
#     print("target_value: ", target_value)

# Save image
PIL.Image.fromarray(formatted_output_value_uint8[0], 'RGB').save('./results/recovery.png')
np.save(os.path.join('./results/recovery.npy'), z_value)


<tf.Variable 'recovery_scope/target_image:0' shape=(1024, 1024, 3) dtype=float32_ref>
Tensor("ExpandDims:0", shape=(1, 1024, 1024, 3), dtype=float32)
<tf.Variable 'recovery_scope/learnable_latents:0' shape=(1, 512) dtype=float32_ref>
Tensor("Gs_1/images_out:0", shape=(1, 3, 1024, 1024), dtype=float32)
Tensor("add:0", shape=(1, 1024, 1024, 3), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Tensor("mean_squared_error/value:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_recovery_scope/learnable_latents/ApplyGradientDescent"

-------
0
Loss:  11389.93
-------
1
Loss:  4418.748
-------
2
Loss:  3400.1702
-------
3
Loss:  3318.0762
-------
4
Loss:  3404.1287
-------
5
Loss:  5238.261
-------
6
Loss:  10704.226
-------
7
Loss:  14634.177
-------
8
Loss:  4497.3535
-------
9
Loss:  2984.5986
-------
10
Loss:  2802.1045
-------
11
Loss:  2666.787
-------
12
Loss:  2583.7456
-------
1

-------
249
Loss:  941.7828
-------
250
Loss:  941.3311
-------
251
Loss:  941.1744
-------
252
Loss:  941.1215
-------
253
Loss:  940.85284
-------
254
Loss:  940.81805
-------
255
Loss:  940.8425
-------
256
Loss:  941.31177
-------
257
Loss:  940.37695
-------
258
Loss:  940.6763
-------
259
Loss:  939.9746
-------
260
Loss:  940.0402
-------
261
Loss:  939.4082
-------
262
Loss:  939.6165
-------
263
Loss:  939.0739
-------
264
Loss:  939.9222
-------
265
Loss:  938.99066
-------
266
Loss:  939.0937
-------
267
Loss:  938.4118
-------
268
Loss:  938.6909
-------
269
Loss:  938.09
-------
270
Loss:  938.3732
-------
271
Loss:  937.8863
-------
272
Loss:  937.7695
-------
273
Loss:  937.44354
-------
274
Loss:  937.53375
-------
275
Loss:  936.9048
-------
276
Loss:  937.3842
-------
277
Loss:  936.6622
-------
278
Loss:  936.7175
-------
279
Loss:  936.1169
-------
280
Loss:  936.47876
-------
281
Loss:  936.21893
-------
282
Loss:  936.514
-------
283
Loss:  935.9431
-------
284
Lo

KeyboardInterrupt: 

In [ ]:
# A minimum example to run 
z = tf.Variable(np.random.randn(1, Gs.input_shape[1]), dtype=tf.float32)
output_variable = Gs.get_output_for(z, None, truncation_psi=0.7, randomize_noise=False)
formatted_output_variable = tflib.convert_images_to_uint8(output_variable, drange=[-1,1], nchw_to_nhwc=True)
print(output_variable)
sess = tf.get_default_session()
in_expr = [z]
mb_in = [np.random.randn(1, Gs.input_shape[1])]
formmated_output_value = sess.run(formatted_output_variable, feed_dict=dict(zip(in_expr, mb_in)))
PIL.Image.fromarray(formmated_output_value[0], 'RGB').save('./results/recovery.png')


In [ ]:
print(Gs)

In [ ]:
from functools import partial

def create_stub(name, batch_size):
    return tf.constant(0, dtype='float32', shape=(batch_size, 0))


def create_variable_for_generator(name, batch_size):
    return tf.get_variable('learnable_dlatents',
                           shape=(batch_size, 18, 512),
                           dtype='float32',
                           initializer=tf.initializers.random_normal())


initial_dlatents = np.zeros((1, 18, 512))
Gs.components.synthesis.run(initial_dlatents,
                               randomize_noise=False, minibatch_size=1,
                               custom_inputs=[partial(create_variable_for_generator, batch_size=1),
                                              partial(create_stub, batch_size=1)],
                               structure='fixed')

sess = tf.get_default_session()
graph = tf.get_default_graph()




In [ ]:
vs = tf.global_variables()
for v in vs:
    print(v)
    
Gs.print_layers()

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [ ]:
print(sess.run(tf.report_uninitialized_variables()))